
<p><img alt="Colaboratory logo" height="45px" src="https://www.quantreo.com/wp-content/uploads/2021/04/cropped-Logo_Quantreo_transparent.png" align="left" hspace="10px" vspace="0px"></p>

# Hyper Paramaters optimization
The objective is to allow you to find the optimal parameters of our strategy.

<br> 
### Content 
* Heatmap parameter selection
* Statistics parameter selection
* Create a function

### Environnement

In [ ]:
!pip install ta
!pip install yfinance

In [ ]:
import pandas as pd
import ta
import yfinance as yf

import seaborn as sns

import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

from matplotlib import cycler
colors = cycler('color',
                ['#669FEE', '#66EE91', '#9988DD',
                 '#EECC55', '#88BB44', '#FFBBBB'])
plt.rc('figure', facecolor='#313233')
plt.rc('axes', facecolor="#313233", edgecolor='none',
       axisbelow=True, grid=True, prop_cycle=colors,
       labelcolor='gray')
plt.rc('grid', color='474A4A', linestyle='solid')
plt.rc('xtick', color='gray')
plt.rc('ytick', direction='out', color='gray')
plt.rc('legend', facecolor="#313233", edgecolor="#313233")
plt.rc("text", color="#C9C9C9")
plt.rc('figure', facecolor='#313233')

### Functions

In [ ]:
def RSI(val,neutral, window):
  """ 
        ------------------------------------------------------------------------------
        | Output: The function gives the returns of RSI strategy                     |
        ------------------------------------------------------------------------------
        | Inputs: -val (type dataframe pandas): Entry values of the stock            |
        |         -neutral (float): Value of neutrality, i.e. no action zone         |
        |         -window (float): rolling period for RSI                            |
        ------------------------------------------------------------------------------
  """

  # Print Error if there is no column Adj Close in the dataframe
  if "Adj Close" not in val.columns:
    ValueError("We need have a columns name Adj Close because all computation are about this column")
  
  val["rsi"] = ta.momentum.RSIIndicator(f["Adj Close"], window=window).rsi()


  
  """ Long buy Signal """
  # We put threshold
  overbuy = 70
  neutral_buy = 50 + neutral

  # Put nan values for the signal long columns
  val["signal_long"] = np.nan
  val["yersteday_rsi"] = f["rsi"].shift(1)
  # We need define the Open Long signal (RSI yersteday<55 and RSI today>55)
  val.loc[(val["rsi"]>neutral_buy) & (val["yersteday_rsi"]<neutral_buy), "signal_long"] = 1

  # We need define the Close Long signal (RSI yersteday>55 and RSI today<55) False signal
  val.loc[(val["rsi"]<neutral_buy)&(val["yersteday_rsi"]>neutral_buy), "signal_long"] = 0

  # We need define the Close Long signal (RSI yersteday>70 and RSI today<70) Over buy signal
  val.loc[(val["rsi"]<overbuy)&(val["yersteday_rsi"]>overbuy), "signal_long"] = 0



  """Short sell signal """
  # We put threshold
  oversell = 30
  neutral_buy = 50 - neutral

  # Put nan values for the signal short columns
  val["signal_short"] = np.nan
  val["yersteday_rsi"] = val["rsi"].shift(1)
  # We need define the Open Short signal (RSI yersteday>45 and RSI today<45)
  val.loc[(val["rsi"]<neutral_buy) & (val["yersteday_rsi"]>neutral_buy), "signal_short"] = -1

  # We need define the Close Short signal (RSI yersteday<45 and RSI today>45) False signal
  val.loc[(val["rsi"]>neutral_buy)&(val["yersteday_rsi"]<neutral_buy), "signal_short"] = 0

  # We need define the Close Short signal (RSI yersteday<30 and RSI today>30) Over sell signal
  val.loc[(val["rsi"]>oversell)&(val["yersteday_rsi"]<oversell), "signal_short"] = 0


  """ Compute the returns """
  # Compute the percentage of variation of the asset
  val["pct"] = val["Adj Close"].pct_change(1)

  # Compute the positions
  val["Position"] = (val["signal_short"].fillna(method="ffill")+ val["signal_long"].fillna(method="ffill"))

  # Compute the return of the strategy
  val["return"] = val["pct"]*(val["Position"].shift(1))

  return val["return"]

In [ ]:
def drawdown_function(serie):
  
  # We compute Cumsum of the returns
  cum = serie.dropna().cumsum()+1

  # We compute max of the cumsum on the period (accumulate max)
  running_max = np.maximum.accumulate(cum)

  # We compute drawdown
  drawdown  = cum/running_max - 1 
  return drawdown

In [ ]:
def BackTest(serie):

  # Import the benchmarcl
  sp500 = yf.download("^GSPC")["Adj Close"].pct_change(1)
  sp500.columns = ["SP500"]

  val = pd.concat((serie, sp500), axis=1).dropna()
  
  # Compute the drawdown
  drawdown = drawdown_function(serie)
  max_drawdown = -np.min(drawdown)

  # Put a subplots
  fig, (cum, dra) = plt.subplots(1,2, figsize=(15,6))
  
  # Put a Suptitle
  fig.suptitle("Backtesting", size=20)

  # Put the cumsum
  cum.plot(serie.cumsum(), color="#39B3C7")
  cum.plot(val["Adj Close"].cumsum(), color="#B85A0F")
  cum.legend(["Portfolio", "SP500"])
  # Set individual title
  cum.set_title("Cumulative Return", size=13)

  # Put the drawdown
  dra.fill_between(drawdown.index,0,drawdown, color="#C73954", alpha=0.65)

  # Set individual title
  dra.set_title("Drawdown", size=13)

  # Plot the graph
  plt.show()

  # Compute the sortino
  sortino = np.sqrt(252)*serie.mean()/serie.loc[serie<0].std()

  # Compute the beta
  beta = np.cov(val,rowvar=False)[0][1] /np.var(val["Adj Close"].dropna())

  # Compute the alpha
  alpha = 252*serie.mean() - 252*beta*serie.mean()

  # Print the statistics
  print(f"Sortino: {np.round(sortino,3)}")
  print(f"Beta: {np.round(beta,3)}")
  print(f"Alpha: {np.round(alpha,3)}")
  print(f"MaxDrawdown: {np.round(max_drawdown*100,3)} %")


# Section 1: Heatmap parameter selection

In [ ]:
# Import the data


# We set lists for the possible values of neutral and window



# We set matrix with only zeros 


# We wil calculate the return of the strategy for each combinations


    # Compute return strategy

    # Compute annualized sortino


# Adapt the size


# Palette for color


# Plot the heatmap


# Put a title


# Put a xlabel


# Put a xlabel


# Show the graph



In [ ]:
# We create just a function for the previous code
def grid_parameters(f):

  # We set lists for the possible values of neutral and window


  # We set matrix with only zeros 



  # We wil calculate the return of the strategy for each combinaison




      # Compute return strategy


      # Compute annualized sortino




In [ ]:
# We set lists for the possible values of neutral and window



# Set some datasets


# Create the grids



# Create a subplots


# Put a sup title


# Change the color


# Train
# Put train heatmap

# Set a title

# Set a xlabel

# Set a ylabel



# Test
# Put test heatmap

# Set a title

# Set a xlabel

# Set a ylabel


# Show the graph



In [ ]:
# BackTest


# Section 2: Statistics parameter selection

In [ ]:
# statistical slection parameters

# We set lists for the possible values of neutral and window




# Set some Border of the datasets



# Initialize the list



# Loop to add the values in the list



      # Compute the returns



      # Compute the sortino


      # We do list of list to do a dataframe


In [ ]:
# Order by Sortino


    # Take the best




    # Compute the sortino

    # Take best neutral and best window

    # If the Sortino of the train and the test are good we stop the loop


    # If there is no values enought good the put 0 in all values




In [ ]:
# Plot the values



In [ ]:
# BackTest


# Section 3: Create a function to use it in the next sections

In [ ]:
def opt(f):
  # We set lists for the possible values of neutral and window


  # Set some Border of the datasets



  # Initialize the list


  # Loop to add the values in the list


  
        # Compute the returns


        # Compute the sortino


        # We do list of list to do a dataframe



  # Order by Sortino



 

    # Take the best

    # Compute the sortino


    # Take best neutral and best window
 

    # If the Sortino of the train and the test are good we stop the loop




   # If there is no values enought good the put 0 in all values



In [ ]:
# Find optimal parameters


In [ ]:
# BackTest
